In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
torch.cuda.init()
device = torch.device("cuda:0")
import numpy as np

In [2]:
def get_sequence(seq,dictionary):
    return torch.tensor([dictionary.get(i,0) for i in seq], dtype = torch.long)

In [3]:
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm_notebook(f):
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model


In [4]:
# glove_model=loadGloveModel("D:/MyCodes/EMBEDDINGS/glove.840B.300d.txt")

# import pickle
# with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","wb") as fh:
#     pickle.dump(glove_model,fh)
    

In [5]:
import pickle
with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","rb") as fh:
    embed_dict=pickle.load(fh)


In [6]:
def create_labels(data):
    
    
    targets=[]
    
    for ques in tqdm_notebook(data):
        if ques[2].lower() not in targets:
            targets.append(ques[2].lower())
            
    return targets


In [7]:
from tqdm import tqdm_notebook
from tqdm import tqdm

In [8]:
with open("D:/SEM 7/Core Project/Dataset/Datasets/Classification_task/classification_clusters.pickle","rb") as fh:
    total_data=pickle.load(fh)

In [9]:
uniq_targets=create_labels(total_data)
rev_targets=dict((v,k) for k,v in enumerate(uniq_targets))

In [10]:

def return_data(dictionary2,data):
    
    if dictionary2==None:
        print("Dictionary is not passed")
        return 
    
    encoded_totl_data=[]
    questions=[]
    encoded_targets=[]
    
    for ques in tqdm_notebook(data):
        
        if "?" in ques[0]:
            ques[0]=ques[0].replace("?"," ?")

        
        encoded_ques=[word.lower() for word in ques[0].split(" ")]  # Tokenizing and encoding
        encoded_targ=dictionary2[ques[2].lower()] # Encoding targets
        
        encoded_totl_data.append([encoded_ques,encoded_targ])
        questions.append(encoded_ques)
        encoded_targets.append(encoded_targ)
        
        
    return encoded_totl_data,questions,encoded_targets
    
    

In [11]:
encoded_total_data,questions,encoded_targets= return_data(rev_targets,total_data)


In [12]:
vocab=list(embed_dict.keys())
vocab.insert(0,"__unk__")


In [13]:
embedding_dim = 300
hidden_dim = 100


In [14]:
class pos_lstm_model(nn.Module):
    def __init__(self,hidden_dim,embedding_dim,vocab_size,tag_size,batch_size):
        super(pos_lstm_model,self).__init__()
        
#         self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        

        self.layer1 = nn.LSTM(embedding_dim,hidden_dim)
        self.layer2 = nn.Linear(hidden_dim,tag_size)
        
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
#         self.embeddings.weight.data.copy_(torch.from_numpy(embedding_matrix))
    def init_hidden(self):
        return (torch.zeros(1,batch_size,hidden_dim,device=device),torch.zeros(1,batch_size,hidden_dim,device=device))

    def forward(self,sentences):
        
#         embed = self.embeddings(sentence)
#         w2vs=[]
    
        maxbatch=0
        
        for sentence in sentences:
            if maxbatch<len(sentence):
                maxbatch=len(sentence)
        
        w2vs=np.zeros((batch_size,maxbatch,embedding_dim),dtype='double')
        
        
        for k,sentence in enumerate(sentences):
            
            w2v=np.zeros((maxbatch,embedding_dim),dtype='double')
            
            for q,wdr in enumerate(sentence):
                
                v = np.zeros(embedding_dim, dtype='float')

                try:
                    v = embed_dict[wdr]
                except:
                    pass
                w2v[q]=v
        
            
            w2vs[k]=w2v
            

        w2vs=np.array(w2vs)
#         print("Embed shape is",w2vs.shape)
#         word_embed = self.word_embeddings(torch.tensor(w2v).cuda())
        embed=torch.cuda.FloatTensor(w2vs)
        embed=embed.permute(1,0,2)
#         print("Embed shape is",embed.shape)
        
        
        output1,self.hidden = self.layer1(embed,self.hidden)
#         print("Output1 ",output1.shape)
#         print(self.hidden[0].shape)
        
#         print("Hidden state is",self.hidden[0])
        
        output2=self.layer2(self.hidden[0])

        return output2[0]

In [15]:
batch_size=700
model = pos_lstm_model(hidden_dim,embedding_dim,len(vocab),len(uniq_targets),batch_size)
model = model.cuda()
loss_function = nn.CrossEntropyLoss().cuda()
# loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)
# optimizer = optim.RMSprop(model.parameters(),lr=0.01)

In [ ]:
# print(questions[0],questions[1],questions[2])
# model.zero_grad()
# model.hidden = model.init_hidden()
# ans=model([questions[0],questions[1],questions[2]])        
# print(ans.shape)
# print(ans)


In [ ]:
# from torch.utils import data
# trainloader = torch.utils.data.DataLoader(encoded_total_data, batch_size=3,shuffle=False, num_workers=2)



In [16]:
class BatchGenerator(object):

    def __init__(self, data,batch_size):
        self.data = data
        self.batch_size = batch_size
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set

    def generate(self):

#         x = np.zeros((self.batch_size, self.num_steps))
        
        
        

        
        while True:
            x=[]
            y=[]

            finish_flag=False
            for i in range(self.batch_size):
                if self.current_idx + 1 > len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                    finish_flag=True
                    print("End of the data")
                
                x += [self.data[self.current_idx][0]]
                y += [self.data[self.current_idx][1]]
                
                self.current_idx += 1
                
    
            yield x,y
            if finish_flag==True:return


        
    
    

In [17]:

data_generator=BatchGenerator(encoded_total_data, batch_size)

In [18]:
num_steps=int(len(encoded_total_data)/batch_size)

In [19]:
num_steps

633

In [ ]:
from tqdm import tqdm
no_epochs = 1

for i in tqdm_notebook(range(no_epochs),leave=True,desc="Epochs"):
#     if i%10 == 0:
    
    if i>0:
        tqdm.write("Loss is %f"%loss)
    for sent,tags in tqdm_notebook(data_generator.generate(),position=1,desc='questions',leave=True,total=num_steps):
        
        
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        
#         tags_in = torch.tensor((), dtype=torch.int32,device=device)
#         tags_in.new_full((), tags)
        
        
        tags=torch.tensor(tags,dtype=torch.long,device=device)

        pred = model([sent])
        
#         print("Prediction size is",pred.size())

        loss = loss_function(pred,tags)
        loss.backward()
        optimizer.step()
print("done!")        

In [ ]:
torch.save(model,'model{}_epochs.pt'.format(no_epochs))

In [ ]:
total=0
correct=0


with torch.no_grad():
    for sentence, tag in zip(questions,encoded_targets):
    #     flatten_parameters()

        model.zero_grad()

        tag_scores= model(sentence) 

        _, predicted = torch.max(tag_scores.data, 1)
        predicted=predicted.cpu().tolist()
        print(predicted,tag)
        total += 1
